In [1]:
import numpy as np
DTYPE = np.float64
# epsilon = np.finfo(DTYPE).eps
epsilon = 1e-3

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

import pandas as pd

import os

In [2]:
# mol vol.s same for different species

def floryHugginsFreeEnergy(phi:DTYPE, chi:DTYPE):
    res = phi*np.log(phi) + (1-phi)*np.log(1-phi) + chi*phi*(1-phi)
    return res

In [3]:
'''
System Parameters:

Given: the total concentration of component 1 in the system
'''
# PHI1_GLOBALs = [DTYPE(0.5)] # Setting the global value of the first component in the system
PHI1_GLOBALs = DTYPE(np.linspace(epsilon, 1-epsilon, 20)) # Setting the global value of the first component in the system

ETA_GLOBAL = DTYPE(1) # Setting the global system size as 1

'''
Flory Parameter(s)
'''
# chi = DTYPE(1.8) # test chi
CHIs = DTYPE(np.linspace(1, 3, 20))

'''
Compartment Parameters

phi11: concentration of component 1 in compartment 1 -> randomly chosen from a uniform distribution u(0, PHI1_GLOBAL)
phi12: concentration of component 1 in compartment 2
phi21: concentration of component 2 in compartment 1
phi22: concentration of component 2 in compartment 2

eta1: size of compartment 1
'''

# eta1 = np.linspace(0, ETA_GLOBAL, 10, dtype=DTYPE) # Size of compartment 1
# eta1 = DTYPE(np.random.uniform(epsilon, ETA_GLOBAL-epsilon, size=1000))
# eta2 = 1 - eta1

'\nCompartment Parameters\n\nphi11: concentration of component 1 in compartment 1 -> randomly chosen from a uniform distribution u(0, PHI1_GLOBAL)\nphi12: concentration of component 1 in compartment 2\nphi21: concentration of component 2 in compartment 1\nphi22: concentration of component 2 in compartment 2\n\neta1: size of compartment 1\n'

In [ ]:
'''
Free energies

f1: free energy of compartment 1
f2: free energy of compartment 2

f_total: total free energy of the system
'''

for chi in CHIs:
    for PHI1_GLOBAL in PHI1_GLOBALs: # Initializing a different system with different global concentration of component 1
        acceptable_phi11s = []
        acceptable_eta1s = []
        acceptable_fs = []
        
        for eta_1 in DTYPE(np.linspace(0+epsilon, 1-epsilon, 1000)): # Setting the compartment 1 size for the system
            for phi_11 in DTYPE(np.linspace(0+epsilon, 1-epsilon, 1000)): # Setting the concentration of component 1 in the first compartment
                phi_12 = (PHI1_GLOBAL - phi_11*eta_1)/(1-eta_1) # Computing phi_12 according to the constraint
                
                flag = 0
                
                if (phi_12 >= 0 and phi_12 <= 1): # Checking if the computed value of phi_12 is physically acceptable
                    flag = 1
                    f1 = floryHugginsFreeEnergy(phi_11, chi)
                    f2 = floryHugginsFreeEnergy(phi_12, chi)
    
                    f = eta_1*f1 + (1-eta_1)*f2
    
                    acceptable_phi11s.append(phi_11)
                    acceptable_eta1s.append(eta_1)
                    acceptable_fs.append(f)
                    
                # print(f"{eta_1:.3f}, {phi_11:.3f}, {phi_12:.3f}, {f1:.3f}, {f2:.3f}, {f:.3f}", flag) # Get an intuition about how many solutions are rejectede
        
        df = pd.DataFrame()
        df["phi11"] = acceptable_phi11s
        df["eta1"] = acceptable_eta1s
        df["F"] = acceptable_fs
        df["log|F|"] = np.log(np.abs(df["F"]))
    
    
    
        fig = plt.figure(figsize=plt.figaspect(0.5))
    
        ax = fig.add_subplot(1, 2, 1, projection='3d')
        # sizes = (df['F'] + df['F'].min()) / (df['F'].max() + df['F'].min())*5  # Normalized
        ax.scatter(df['phi11'], df['eta1'], df['F'], c=df['F'], cmap='viridis', s = 1)
        ax.set_xlabel(r"$\phi_{11}$")
        ax.set_ylabel(r"$\eta_1$")
        ax.set_zlabel('F')
    
        ax = fig.add_subplot(1, 2, 2)
    
        title = r"$\phi_1^{\text{global}}$ = " + f"{PHI1_GLOBAL:.3f}" + "\n" + r"$\chi = $" + f"{chi:.3f}"
        fig.suptitle(title)
        pivot = df.pivot(index='eta1', columns='phi11', values='F')
     
        sns.heatmap(pivot, cmap='viridis', ax = ax)
    
        ax.set_xticks(np.arange(0, len(pivot.columns), 100))
        ax.set_xticklabels( [f"{val:.3f}" for val in pivot.columns[::100]], rotation=90)
        ax.set_xlabel(r"$\phi_{11}$")
    
        ax.set_yticks(np.arange(0, len(pivot.index), 100))
        ax.set_yticklabels([f"{val:.3f}" for val in pivot.index[::100]], rotation=0);
        ax.set_ylabel(r"$\eta_1$")
        
        fig.tight_layout()

        output_filepath = f"data/chi-{chi:.3f}"
        output_filename = f"phi_g-{PHI1_GLOBAL:.3f}.png"

        if not os.path.exists(output_filepath):
            os.makedirs(output_filepath)
        
        file = os.path.join(output_filepath, output_filename)     
        
        plt.savefig(file, dpi = 400)
        plt.close()

        print(f"Saved @ {file}")

Saved @ data/chi-1.000/phi_g-0.001.png
Saved @ data/chi-1.000/phi_g-0.112.png
Saved @ data/chi-1.000/phi_g-0.223.png
Saved @ data/chi-1.000/phi_g-0.334.png
Saved @ data/chi-1.000/phi_g-0.445.png
